In [1]:
import json
import PyPDF2
from pathlib import Path
import os
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

In [2]:
ENV_DIR = Path().absolute().parent / ".env"
DEV_ENV_FILE_PATH = ENV_DIR / "dev.env"

In [3]:
load_dotenv(DEV_ENV_FILE_PATH, override=True)

True

In [4]:
ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT_SLN')
KEY = os.getenv('AZURE_OPENAI_API_KEY_SLN')
API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')

In [5]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        text = ""
        for page in range(reader.numPages):
            text += reader.getPage(page).extractText()
    return text

In [6]:
def analyze_text(question, text):
    credential = AzureKeyCredential(KEY)
    client = TextAnalyticsClient(endpoint=ENDPOINT, credential=credential)

    request_data = {
        "question": question,
        "context": text
    }
    headers = {
        "Ocp-Apim-Subscription-Key": KEY,
        "Content-Type": "application/json"
    }

    response = client.request(
        method="POST",
        path="/qna/{}/query".format(API_VERSION),
        headers=headers,
        json_body=request_data
    )

    result = response.json()
    answer = result['answers'][0]['answer'] if result['answers'] else ""
    return answer

In [7]:
def answer_question(question, pdf_path):
    # Extract text from PDF
    text = extract_text_from_pdf(pdf_path)
    
    # Analyze text using Azure Cognitive Services and get the answer
    answer = analyze_text(question, text)
    
    return answer

In [9]:
# path_is = Path().absolute().parent / "data"
# path_pdf = ENV_DIR / "war.pdf"
pdf_file_path = Path().absolute().parent / "data" / "aboutwar.pdf"
def main():
    pdf_path = pdf_file_path
    user_question = "What is the capital of France?"
    
    answer = answer_question(user_question, pdf_path)
    print("Answer:", answer)


if __name__ == '__main__':
    main()

DeprecationError: PdfFileReader is deprecated and was removed in PyPDF2 3.0.0. Use PdfReader instead.